<a href="https://colab.research.google.com/github/Gauthameshwar/RAQS_QuantumInspiredAlgos/blob/master/notebooks/RSVDnDataCompression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Randomized SVD → Tensor Networks for Data Compression

**Notebook author:** Gauthameshwar Sundaravadivel

---


> **You’ll learn**
> - **Randomized SVD (RSVD)**: faster than classical SVD; power/subspace iterations for accuracy.
> - **Tensor-Train (TT/MPS)**: compress multiway data with small ranks.
> - **Case study**: gene-expression compression with TT.
>
> **Prerequisites:** Linear algebra, basic Julia.


---

<details>
  <summary>Outline (click to expand)</summary>

  1. SVD recap → low-rank ≈ compression  
  2. RSVD algorithm, complexity & error, power iterations  
  3. Tensor networks (MPS/TT) without physics baggage  
  4. TT-SVD (Oseledets) and where RSVD slots in  
  5. Gene-expression compression mini-project  
</details>

---

**Prerequisites:** Linear algebra, basic Julia.

## 0. Setup

We borrow the package deps for this notebook from the home GitHub repo of this notebook, and activate it before diving into the exercises.

In [9]:
const USER = "Gauthameshwar"
const REPO = "RAQS_QuantumInspiredAlgos"
const BRANCH = "master"

cd("/content")

if !isdir(REPO)
  run(`git clone --depth 1 -b $BRANCH https://github.com/$USER/$REPO.git`)
end

import Pkg
proj = joinpath("/content", REPO, "env")  # where Project.toml lives
isfile(joinpath(proj, "Project.toml")) || error("Project.toml not found: $proj")
Pkg.activate(proj)
Pkg.instantiate()   # uses Manifest.toml if present; pins exact versions

# proj = joinpath(pwd(), "notebooks", "env") # e.g., .../notebooks/env/Project.toml
# isdir(proj) || error("Project path not found: $proj") Pkg.activate(proj)


  Activating project at `/content/RAQS_QuantumInspiredAlgos/env`


In [10]:
using LinearAlgebra, Plots, Random, Logging

## 1. Randomised SVD

---
> **Motivation**
>
> Many real datasets(images, audio, experimental measurements) can often be described well with only a **few important directions (singular vectors)**. That means the matrix is **approximately low rank**. A full SVD finds _all_ directions and can be slow on big matrices. RSVD focuses on the important part first, so it's **much faster** when you only need the top $k$ components

---

> **Idea in a nutshell**
>
> Instead of SVD-ing a big matrix $A$, we first find a small **orthonormal basis $Q$** that spans (most of) the columns of $A$. Then we SVD a **small matrix $B=Q^TA$** and lift the result back with $Q$.

---

Quick notation and definitions:

- Matrix $A\in \mathbb{R}^{m\times n}$: A grid of numbers with $m$ rows and $n$ columns.
- Rank $k$: How many important directions (singular vectors) we wish to keep.
- Orthonormal basis $Q\in \mathbb{R}^{m\times l}$: Columns of $Q$ are unit-length and mutually perpendicular.
- SVD: Factorisation of the matrix into $A = U \Sigma V^T$ such that $U \in \mathbb{R}^{m\times m}$ and $V \in \mathbb{R}^{n\times n}$ are orthonormal matrices, and $\Sigma \in \mathbb{R}^{m\times n}$ is a diagonal matrix with the singular values as its diagonal entries (from largest to smallest in magnitude).
- Oversampling parameter $p$: Additional number of rank into the sampling so we don't miss important information in the last singular vectors (optimal values around 5-20).
- Power iterations $q$: Extra passes of the subspace so we sharpen the accuracy of the singular values and its vectors.


In a standard SVD, given a matrix $A\in \mathbb{R}^{m\times n}$, we compute
$$
A = U \Sigma V^T.
$$
The best algorithm of SVD in Julia is provided by `LinearAlgebra`'s `LAPACK` support which has the runtime complexity of
$$
O(mn\times \text{min}(m, n)).
$$
This can get slow when both $m$ and $n$ are large. In the worst case scenario, where we have a large square matrix, the complexity scales as $O(n^3)$.

Also, the memory complexity of performing the SVD goes as
$$O(mn + m^2 + n^2),$$ which is quadratic in the largest size.

### The RSVD algorithm

> Goal: To get a good rank‑$k$ approximation quickly by working with a randomised subspace.

Stage A — Find a good basis

1. Make a random test matrix  with $\Omega \in \mathbb{R}^{n\times l}$ with $l = k + p$.

2. Form a sample of ’s column space: $Y = A\Omega \in \mathbb{R}^{m\times l}$.

3. Compute a thin QR of $Y$: $Y = QR$. Keep $Q \in \mathbb{R}^{m\times l}$.

Stage B — Work in small coordinates

4. Compress $A$ to a small matrix: $B = Q^TA \in \mathbb{R}^{l\times n}$.

5. Do SVD on the small matrix: $B = \tilde{U}\Sigma V^T$.

6. Map back: $U \approx Q \tilde{U}$.

7. Truncate to rank $k$: keep the first $k$ columns of $U$ and $V$, and the first $k$ values in $\Sigma$.

Optional accuracy boost (power iterations): if the singular values don’t drop quickly, repeat extra multiplies $q$ times so the important directions stand out more.


In [11]:
function rsvd(A::AbstractMatrix{<:Union{Real, Complex}}, l::Int, p::Int=10)
    m, n = size(A)
    @info "Starting Randomized SVD" DataSize=(m, n) RequestedSingularValues=l OversamplingParameter=p TotalColumnsForSampling=(l+p)

    # sample a random matrix Y in the subspace of A
    @info "1. Sampling random matrix Y..."
    time_Y = @elapsed Y = A * randn(n, l + p)
    @info "Y sampling completed" Time=time_Y

    # form the sample matrix B
    @info "2. Performing QR factorization of Y and forming sample matrix B..."
    time_B = @elapsed Q = qr(Y).Q[:, 1:min(l+p, size(qr(Y).Q, 2))]
    B = Q' * A
    @info "B formation completed" Time=time_B

    # compute the SVD of B
    @info "3. Computing the SVD of B..."
    time_svd_B = @elapsed Ũ, D, V = svd(B)
    @info "SVD of B completed" Time=time_svd_B

    # compute the approximate singular vectors of A
    @info "4. Computing the approximate singular vectors of A..."
    time_U = @elapsed U = Q * Ũ
    @info "Approximate U computation completed" Time=time_U

    @info "Randomized SVD completed"

    return U, D, V
end


rsvd (generic function with 2 methods)

### Runtime and memory complexity of RSVD
1. Runtime
- The heavy work in this algorithm is done in the largest matrix multiplications: $Y = A \Omega$. When the matrix is dense (worst-case scenario), it has a rough cost of $O(mnl)$.

- The QR and SVD has $O(ml^2 + nl^2)$ together. Thus, they are not the bottleneck of this algorithm for small values of $l$.

2. Memory
- Excluding the storing of the original data $A$, the memory complexity of RSVD is $O((m + n)l)$.
- This makes it more memory efficient for larger data sizes than SVD since it scales linearly in the largest dimension.

In [15]:
# Generate a random matrix to test the RSVD
m, n = 10000, 10000
A = randn(m, n)

rsvd(A, 20);


┌ Info: Starting Randomized SVD
│   DataSize = (10000, 10000)
│   RequestedSingularValues = 20
│   OversamplingParameter = 10
└   TotalColumnsForSampling = 30
[ Info: 1. Sampling random matrix Y...
┌ Info: Y sampling completed
└   Time = 0.404582439
[ Info: 2. Performing QR factorization of Y and forming sample matrix B...
┌ Info: B formation completed
└   Time = 0.01548268
[ Info: 3. Computing the SVD of B...
┌ Info: SVD of B completed
└   Time = 0.024392761
[ Info: 4. Computing the approximate singular vectors of A...
┌ Info: Approximate U computation completed
└   Time = 0.00158533
[ Info: Randomized SVD completed


### Accuracy of RSVD